In [2]:
import os
import sys
import time
import numpy as np
import pandas as pd
import xarray as xr
from scipy.interpolate import interp1d
import matplotlib
%matplotlib inline
#matplotlib.rcParams['font.size'] = 12
import matplotlib.pyplot as plt
import necstdb

In [3]:
cwd = os.getcwd()
db = necstdb.opendb(cwd)

In [2]:
def conv_xarray(db, topic, name=''):
    array_data = db.open_table(topic).read(astype="array")
    xarray_data = xr.DataArray(array_data['data'], dims=['timestamp'], coords={'timestamp' : array_data['timestamp']}, name=name)
    return xarray_data

def resample_pre(resample_xarray, reference_xarray, index="timestamp", kind="previous", **kwargs):
    resample_func = interp1d(resample_xarray[index].values, resample_xarray.values, kind, **kwargs)
    return resample_func(reference_xarray[index].values)

In [ ]:
power = conv_xarray(db, 'ml2437a')[2:-2]

In [ ]:
slider = conv_xarray(db, 'slider_location_cmd')
vol = conv_xarray(db, 'sis_vol_2l')
vol_cmd = conv_xarray(db, 'sis_vol_2l_cmd')
loatt = conv_xarray(db, 'loatt_2l_cmd')
cur = conv_xarray(db, 'sis_cur_2l')

In [ ]:
data = xr.DataArray(power.values, coords={'timestamp' : power['timestamp'].values, 'sisvol' : ('timestamp', resample_pre(vol, power, kind='linear')), 'sisvol_cmd' : ('timestamp', resample_pre(vol_cmd, power, fill_value="extrapolate")), 'slider' : ('timestamp', resample_pre(slider, power, fill_value="extrapolate")),  'siscur' : ('timestamp', resample_pre(cur, power, kind = 'linear')), 'loatt' : ('timestamp', resample_pre(loatt, power, fill_value="extrapolate"))}, dims='timestamp')

In [ ]:
pd_data = data.to_dataframe("power")

In [ ]:
pd_mean = pd_data.groupby(['loatt', 'sisvol_cmd', 'slider']).median()

In [ ]:
pd_mean

In [ ]:
pd_mean['power_diff'] = pd_mean.groupby(['loatt','sisvol_cmd'])['power'].diff()

In [ ]:
pd_mean

In [ ]:
pd_power = pd_mean[pd_mean['power_diff'].notna()].reset_index()

In [ ]:
pd_power['yfactor']=10**(-pd_power['power_diff']/10)

In [ ]:
pd_power

In [ ]:
pd_power.sort_values("yfactor")[-10:]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
mappable = ax.scatter(pd_power['sisvol_cmd'], pd_power['siscur'], c=pd_power['yfactor'], cmap='jet', s=5)
#ax.set_xlim(0,10)
ax.set_xlabel('Voltage ( mV )')
ax.set_ylabel('Current ( uA )')
bar = fig.colorbar(mappable)
bar.set_label('Y-factor')
#plt.savefig('yfactor.pdf')